## Evaluation with human judgements from SimpEval

In [45]:
!pip install evaluate
!pip install sacrebleu
!pip install pandarallel

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.

In [46]:
import pickle
import numpy as np
from labeling_functions import get_all_lfs
import pandas as pd
import os
import tqdm

from pandarallel import pandarallel
pandarallel.initialize()
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis

import warnings

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [49]:
# Metrics
from evaluate import load
sari = load("sari")

from usb_utils import bertscore
import textstat
fkgl = textstat.flesch_kincaid_grade

In [36]:
# Datasets
df_simpeval_2022 = pd.read_csv("/workspace/datasets/simpeval/LENS/data/simpeval_2022.csv")
df_simpeval_DA = pd.read_csv("/workspace/datasets/simpeval/LENS/data/simpDA_2022.csv")
df_simpeval_likert = pd.read_csv("/workspace/datasets/simpeval/LENS/data/simplikert_2022.csv")

In [88]:
df_simpeval_DA_dict = {}
for id in df_simpeval_DA["Input.id"].unique():
    df_simpeval_DA_dict[id] = {
    "Input.original": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id].iloc[0]["Input.original"],
    "Input.simplified": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id].iloc[0]["Input.simplified"],
    "Input.system": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id].iloc[0]["Input.system"],
    "Answer.adequacy": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id]["Answer.adequacy"].mean(),
    "Answer.fluency": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id]["Answer.fluency"].mean(),
    "Answer.simplicity": df_simpeval_DA.loc[df_simpeval_DA["Input.id"] == id]["Answer.simplicity"].mean()
    }
df_simpeval_DA_mean = pd.DataFrame(df_simpeval_DA_dict).T

df_simpeval_likert_dict = {}
for id in df_simpeval_likert["Input.id"].unique():
    df_simpeval_likert_dict[id] = {
    "Input.original": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id].iloc[0]["Input.original"],
    "Input.simplified": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id].iloc[0]["Input.simplified"],
    "Input.system": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id].iloc[0]["Input.system"],
    "Answer.adequacy": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id]["Answer.adequacy"].mean(),
    "Answer.fluency": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id]["Answer.fluency"].mean(),
    "Answer.simplicity": df_simpeval_likert.loc[df_simpeval_likert["Input.id"] == id]["Answer.simplicity"].mean()
    }
df_simpeval_likert_mean = pd.DataFrame(df_simpeval_likert_dict).T

In [91]:
df_simpeval_2022

,original_id,original,generation,system,sentence_type,rating_1,rating_2,rating_3,rating_1_zscore,rating_2_zscore,rating_3_zscore,sari,FKGL
0,37,"According to the eyewitness account, the shoot...",The eyewitness reported that the shooter becam...,GPT-3-zero-shot,Splittings,100.0,70.0,70.0,1.361822,0.034570,0.344509,66.666667,4.4
1,7,After defeating PSD candidate Viorica Dăncilă ...,"After winning in 2019, his second term was mar...",Human 1 Writing,Deletions,95.0,80.0,100.0,1.010140,0.601185,2.243569,66.666667,14.2
2,47,"The sentence included a fine of $400, or $100 ...","They imposed a fine of $400, or $100 per count...",GPT-3-few-shot,Paraphrases,61.0,70.0,50.0,-1.381293,0.034570,-0.921531,66.666667,8.3
3,50,"On November 9, Fetterman was projected as the ...",The media projected Fetterman to win the elect...,GPT-3-zero-shot,Paraphrases,58.0,90.0,98.0,-1.592302,1.167799,2.116965,66.666667,8.8
4,9,"Musk also announced the creation of a ""content...","Musk announced the formation of a ""content mod...",GPT-3-zero-shot,Splittings,92.0,95.0,60.0,0.799132,1.451107,-0.288511,66.666667,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,11,The last president to run after leaving office...,The last president to run after leaving office...,T5-3B,Deletions,63.0,80.0,65.0,-1.240621,0.601185,0.027999,75.000000,6.4
356,17,"Unlike the first film adaptation, in which dir...","Unlike the first movie, Barbeau-Lavalette did ...",Muss,Splittings,80.0,74.0,50.0,-0.044904,0.261216,-0.921531,66.666667,7.2
357,45,The film received positive reviews from critic...,The film received overall positive reviews wit...,Human 1 Writing,Splittings,95.0,90.0,80.0,1.010140,1.167799,0.977529,66.666667,9.2
358,58,The Creative Gene is an autobiographical book ...,The Creative Gene is a book written by Kojima ...,Human 1 Writing,Splittings,100.0,90.0,60.0,1.361822,1.167799,-0.288511,66.666667,4.4


In [90]:
df_simpeval_2022["sari"] = df_simpeval_2022.progress_apply(lambda row:  sari.compute(sources=[row["original"]], predictions=[row["generation"]], references=[[]])["sari"] , axis = 1)
df_simpeval_DA_mean["sari"] = df_simpeval_DA_mean.progress_apply(lambda row:  sari.compute(sources=[row["Input.original"]], predictions=[row["Input.simplified"]], references=[[]])["sari"] , axis = 1)
df_simpeval_likert_mean["sari"] = df_simpeval_likert_mean.progress_apply(lambda row:  sari.compute(sources=[row["Input.original"]], predictions=[row["Input.simplified"]], references=[[]])["sari"] , axis = 1)

# BertScore
#df_simpeval_2022["BERTscore"] = df_simpeval_2022.progress_apply(lambda row: bertscore(row["original"], row["generation"]) , axis = 1)
df_simpeval_DA_mean["BERTscore"] = df_simpeval_DA_mean.progress_apply(lambda row: bertscore(row["Input.original"], row["Input.simplified"]) , axis = 1)
#df_simpeval_likert_mean["BERTscore"] = df_simpeval_likert_mean.progress_apply(lambda row: bertscore(row["Input.original"], row["Input.simplified"]) , axis = 1)

# FKGL
df_simpeval_2022["FKGL"] = df_simpeval_2022.progress_apply(lambda row: fkgl(row["generation"]) , axis = 1)
df_simpeval_DA_mean["FKGL"] = df_simpeval_DA_mean.progress_apply(lambda row: fkgl(row["Input.simplified"]) , axis = 1)
df_simpeval_likert["FKGL"] = df_simpeval_likert.progress_apply(lambda row: fkgl(row["Input.simplified"]) , axis = 1)

  0%|          | 0/60 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  3%|▎         | 2/60 [00:18<09:05,  9.41s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  5%|▌         | 3/60 [00:28<09:06,  9.59s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  7%|▋         | 4/60 [00:41<10:12, 10.94s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1080/1080 [00:00<00:00, 72209.53it/s]


In [94]:
df_simpeval_DA_mean

,Input.original,Input.simplified,Input.system,Answer.adequacy,Answer.fluency,Answer.simplicity,sari,BERTscore,FKGL
40,"On the fifth day of flight, November 20, 2022,...","On November 20, 2022, the Orion spacecraft sta...",GPT-3-few-shot,89.5,97.111111,73.333333,66.666667,0.586026,6.4
11,The last president to run after leaving office...,The last president who ran after leaving the p...,Human 1 Writing,72.666667,90.777778,86.277778,66.666667,0.645294,9.9
53,The complainant claimed that he had not renoun...,The complaint said he did not renounce his Nep...,Human 2 Writing,92.333333,95.388889,73.222222,66.666667,0.614902,10.3
30,The club said on social media that customers s...,The club announced on social media that custom...,GPT-3-zero-shot,92.0,92.055556,68.222222,66.666667,0.726086,10.5
1,Bone has published numerous books including Fa...,Bone has published many books including Fantas...,Muss,87.333333,85.444444,73.5,66.666667,0.800024,9.5
46,The film has grossed over $552 million worldwi...,"The film earned over $552 million globally, ma...",GPT-3-few-shot,80.055556,94.166667,76.111111,66.666667,0.804261,7.2
45,The film received positive reviews from critic...,"Film critics praised Coogler’s direction, acti...",Human 2 Writing,86.111111,90.611111,73.944444,66.666667,0.488568,9.1
39,The activists called out to the Russian delega...,The activists targeted the Russian delegation ...,Human 2 Writing,88.0,90.833333,72.833333,66.666667,0.530162,7.6
21,"In a difficult situation, it encouraged him to...","In a difficult situation, it encouraged him to...",Muss,85.5,79.277778,76.333333,66.666667,0.644130,10.9
35,According to anonymous sources cited by The Wa...,"According to anonymous sources, Sam Bankman-Fr...",Human 1 Writing,91.444444,89.666667,74.111111,75.000000,0.572698,13.0


In [93]:
# calculate human scores for similarity measurement
df_simpeval_2022["human_mean"] = df_simpeval_2022.progress_apply(lambda row:  np.mean(np.array([row["rating_1"], row["rating_2"], row["rating_3"]]))  , axis = 1) 

100%|██████████| 360/360 [00:00<00:00, 45587.51it/s]


In [12]:
x = np.array(df_simpeval_2022["human_mean"].to_list())
y = np.array(df_simpeval_2022["sari"].to_list())

from scipy import stats
res = stats.kendalltau(x, y)
res.statistic

-0.1462688364189557

In [97]:
pearson_r = {"simpeval_DA": {"BERTscore":stats.pearsonr(df_simpeval_DA_mean["Answer.simplicity"].to_list(), df_simpeval_DA_mean["BERTscore"].to_list()), "sari":stats.pearsonr(df_simpeval_DA_mean["Answer.simplicity"].to_list(), df_simpeval_DA_mean["sari"].to_list()), "fkgl":stats.pearsonr(df_simpeval_DA_mean["Answer.simplicity"].to_list(), df_simpeval_DA_mean["FKGL"].to_list())}}


In [98]:
pearson_r

{'simpeval_DA': {'BERTscore': PearsonRResult(statistic=-0.15447831661261757, pvalue=0.23859629335543914),
  'sari': PearsonRResult(statistic=-0.037139832464265526, pvalue=0.7781510258676065),
  'fkgl': PearsonRResult(statistic=-0.0935125777962679, pvalue=0.4772903066460549)}}

In [15]:
from bats_label_model import BatsModel

In [99]:
model = BatsModel("MTurkSF")

100%|██████████| 100/100 [00:00<00:00, 259.44epoch/s]
/workspace/pruning_lfs.py:52: RuntimeWarning: invalid value encountered in scalar divide
  merged_data.append([index, polarity, cov_simp, cov_src, precision, 1 - precision, cov_simp+cov_src, abs(cov_simp - cov_src), abs(cov_simp - cov_src)/(cov_simp+cov_src)])
/usr/local/lib/python3.8/site-packages/snorkel/labeling/model/label_model.py:387: RuntimeWarning: divide by zero encountered in divide
  return np.clip(accs / self.coverage, 1e-6, 1.0)


In [100]:
from pruning_lfs import prune_lfs as pruner

In [101]:
lfs = pruner()

/workspace/pruning_lfs.py:52: RuntimeWarning: invalid value encountered in scalar divide
  merged_data.append([index, polarity, cov_simp, cov_src, precision, 1 - precision, cov_simp+cov_src, abs(cov_simp - cov_src), abs(cov_simp - cov_src)/(cov_simp+cov_src)])


In [102]:
len(lfs)

328

In [103]:
model.calc_score("A taxonomy of inverted parantheses, if carried out in the approriate manner, does not cause severe harm to indistinguishable mice and other rodents.")

0.0

In [104]:
df_test = df_simpeval_2022[:50]

In [105]:
df_test["test_MTurkSF_orig"] = df_test.progress_apply(lambda row: model.calc_score(row["original"]) , axis = 1)

100%|██████████| 50/50 [00:05<00:00,  9.03it/s]
/tmp/ipykernel_361/2184013543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["test_MTurkSF_orig"] = df_test.progress_apply(lambda row: model.calc_score(row["original"]) , axis = 1)


In [106]:
df_test

,original_id,original,generation,system,sentence_type,rating_1,rating_2,rating_3,rating_1_zscore,rating_2_zscore,rating_3_zscore,sari,FKGL,human_mean,test_MTurkSF_orig
0,37,"According to the eyewitness account, the shoot...",The eyewitness reported that the shooter becam...,GPT-3-zero-shot,Splittings,100.0,70.0,70.0,1.361822,0.034570,0.344509,66.666667,4.4,80.000000,1.000
1,7,After defeating PSD candidate Viorica Dăncilă ...,"After winning in 2019, his second term was mar...",Human 1 Writing,Deletions,95.0,80.0,100.0,1.010140,0.601185,2.243569,66.666667,14.2,91.666667,1.000
2,47,"The sentence included a fine of $400, or $100 ...","They imposed a fine of $400, or $100 per count...",GPT-3-few-shot,Paraphrases,61.0,70.0,50.0,-1.381293,0.034570,-0.921531,66.666667,8.3,60.333333,0.000
3,50,"On November 9, Fetterman was projected as the ...",The media projected Fetterman to win the elect...,GPT-3-zero-shot,Paraphrases,58.0,90.0,98.0,-1.592302,1.167799,2.116965,66.666667,8.8,82.000000,0.000
4,9,"Musk also announced the creation of a ""content...","Musk announced the formation of a ""content mod...",GPT-3-zero-shot,Splittings,92.0,95.0,60.0,0.799132,1.451107,-0.288511,66.666667,8.8,82.333333,1.000
5,45,The film received positive reviews from critic...,Critics praised the film's direction by Coogle...,GPT-3-zero-shot,Splittings,90.0,60.0,55.0,0.658459,-0.532044,-0.605021,66.666667,8.8,68.333333,1.000
6,2,"In Marriages and Infidelities, Oates explicitl...","In Marriages and Infidelities, Oates takes ide...",GPT-3-few-shot,Paraphrases,85.0,60.0,95.0,0.306777,-0.532044,0.984394,66.666667,13.4,80.000000,1.000
7,9,"Musk also announced the creation of a ""content...","Musk announced the formation of a ""content mod...",GPT-3-few-shot,Splittings,60.0,85.0,53.0,-1.451630,0.884492,-0.731625,66.666667,11.9,66.000000,1.000
8,41,These missions will seek to reestablish a huma...,Missions will be launched to return people to ...,GPT-3-zero-shot,Paraphrases,80.0,55.0,80.0,-0.044904,-0.815352,-0.418885,66.666667,16.2,71.666667,0.985
9,20,"Whereas earlier major protests, which were vio...","Earlier major protests, which were violently q...",Muss,Splittings,76.0,88.0,48.0,-0.326249,0.386161,-1.048135,66.666667,10.9,70.666667,1.000


In [25]:
from load_simplicity import load_asset_ds
from load_simplicity import load_metaeval_ds
from load_simplicity import load_questeval_ds
from load_simplicity import load_simpeval_ds

asset_simp = load_asset_ds()
metaeval_simp = load_metaeval_ds()
questeval_simp = load_questeval_ds()
simpeval_simp = load_simpeval_ds()

In [26]:
simpeval_simp

,ds_id,src,simp,simplicityScore,meaningScore,origin,inner_ds
0,SimpEval_22,"On the fifth day of flight, November 20, 2022,...","On November 20, 2022, the Orion spacecraft sta...",93.555556,92.833333,GPT-3-few-shot,"[simpDA_2022, simplikert_2022, simpeval_2022]"
1,SimpEval_22,The last president to run after leaving office...,The last president who ran after leaving the p...,86.888889,77.500000,Human 1 Writing,"[simpDA_2022, simplikert_2022, simpeval_2022]"
2,SimpEval_22,The complainant claimed that he had not renoun...,The complaint said he did not renounce his Nep...,87.777778,56.000000,Human 2 Writing,"[simpDA_2022, simplikert_2022, simpeval_2022]"
3,SimpEval_22,The club said on social media that customers s...,The club announced on social media that custom...,70.111111,85.500000,GPT-3-zero-shot,"[simpDA_2022, simplikert_2022, simpeval_2022]"
4,SimpEval_22,Bone has published numerous books including Fa...,Bone has published many books including Fantas...,77.500000,85.166667,Muss,"[simpDA_2022, simplikert_2022]"
...,...,...,...,...,...,...,...
2766,SimpEval_22,MacGruber starts asking for simple objects to ...,Macgruber starts asking for simple objects to ...,82.400000,-1.000000,new_simple_wiki,[simpeval_past]
2767,SimpEval_22,MacGruber starts asking for simple objects to ...,MacGruber asks for objects to help defuse the ...,91.000000,-1.000000,asset.test.simp,[simpeval_past]
2768,SimpEval_22,MacGruber starts asking for simple objects to ...,MacGruber starts asking for simple objects. Ma...,46.200000,-1.000000,SEMoses_all,[simpeval_past]
2769,SimpEval_22,MacGruber starts asking for simple objects to ...,MacGruber starts asking for simple objects to ...,86.800000,-1.000000,new_asset.test.simp.second,[simpeval_past]


In [27]:
from usb_utils import bertscore

In [28]:
bertscore(simpeval_simp.loc[12]["src"], simpeval_simp.loc[12]["simp"])

0.6638270020484924

In [29]:
from usb_utils import mpire_score

In [30]:
mpire_score(simpeval_simp.loc[12]["src"], simpeval_simp.loc[12]["simp"])

0.9467238009813017

In [31]:
simpeval_simp.loc[12]["meaningScore"]

93.0

In [32]:
from sklearn.metrics.cluster import adjusted_rand_score

In [33]:
cluster_manual = pickle.load(open("/workspace/datasets/clustering_manual_LFs.pkl", "rb"))
cluster_automatic = pickle.load(open("/workspace/datasets/auto_cluster_dic.pkl", "rb"))

auto = []
manual = []

for k in cluster_manual.keys():
    auto.append(cluster_automatic[k])
    manual.append(cluster_manual[k])
    print(k,auto[-1], manual[-1])

adjusted_rand_score(manual, auto)

lf_words_cnt_wcount=15_SIMPLE 0 0
lf_words_cnt_wcount=16_SIMPLE 0 0
lf_words_cnt_wcount=17_SIMPLE 0 0
lf_words_cnt_wcount=18_SIMPLE 0 0
lf_words_cnt_wcount=19_SIMPLE 2 0
lf_words_cnt_wcount=20_SIMPLE 2 0
lf_words_cnt_wcount=21_SIMPLE 2 0
lf_words_cnt_wcount=22_SIMPLE 2 0
lf_words_cnt_wcount=23_SIMPLE 2 0
lf_words_cnt_wcount=24_SIMPLE 2 0
lf_words_cnt_wcount=20_NOT_SIMPLE 1 0
lf_words_cnt_wcount=21_NOT_SIMPLE 1 0
lf_words_cnt_wcount=22_NOT_SIMPLE 1 0
lf_words_cnt_wcount=23_NOT_SIMPLE 1 0
lf_words_cnt_wcount=24_NOT_SIMPLE 1 0
lf_words_cnt_wcount=25_NOT_SIMPLE 1 0
lf_words_cnt_wcount=26_NOT_SIMPLE 1 0
lf_words_cnt_wcount=27_NOT_SIMPLE 1 0
lf_words_cnt_wcount=28_NOT_SIMPLE 1 0
lf_words_cnt_wcount=29_NOT_SIMPLE 0 0
lf_words_cnt_wcount=30_NOT_SIMPLE 0 0
lf_words_cnt_wcount=31_NOT_SIMPLE 0 0
lf_words_cnt_wcount=32_NOT_SIMPLE 0 0
lf_words_cnt_wcount=33_NOT_SIMPLE 0 0
lf_words_cnt_wcount=34_NOT_SIMPLE 0 0
lf_words_cnt_wcount=35_NOT_SIMPLE 0 0
lf_words_cnt_wcount=36_NOT_SIMPLE 0 0
lf_words_cnt_w

0.07048466331344759

In [34]:
cluster_automatic

{'lf_words_cnt_wcount=15_SIMPLE': 0,
 'lf_words_cnt_wcount=16_SIMPLE': 0,
 'lf_words_cnt_wcount=17_SIMPLE': 0,
 'lf_words_cnt_wcount=18_SIMPLE': 0,
 'lf_words_cnt_wcount=19_SIMPLE': 2,
 'lf_words_cnt_wcount=20_SIMPLE': 2,
 'lf_words_cnt_wcount=21_SIMPLE': 2,
 'lf_words_cnt_wcount=22_SIMPLE': 2,
 'lf_words_cnt_wcount=23_SIMPLE': 2,
 'lf_words_cnt_wcount=24_SIMPLE': 2,
 'lf_words_cnt_wcount=20_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=21_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=22_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=23_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=24_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=25_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=26_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=27_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=28_NOT_SIMPLE': 1,
 'lf_words_cnt_wcount=29_NOT_SIMPLE': 0,
 'lf_words_cnt_wcount=30_NOT_SIMPLE': 0,
 'lf_words_cnt_wcount=31_NOT_SIMPLE': 0,
 'lf_words_cnt_wcount=32_NOT_SIMPLE': 0,
 'lf_words_cnt_wcount=33_NOT_SIMPLE': 0,
 'lf_words_cnt_wcount=34_NOT_SIMPLE': 0,
 'lf_words_cnt_

In [35]:
lf_names = [a.name for a in prune_lfs()]

NameError: name 'prune_lfs' is not defined

In [ ]:
model_lf_weights = {}
for i, weight in enumerate(model.label_model.get_weights()):
    model_lf_weights[lf_names[i]] = weight

/usr/local/lib/python3.8/site-packages/snorkel/labeling/model/label_model.py:387: RuntimeWarning: divide by zero encountered in divide
  return np.clip(accs / self.coverage, 1e-6, 1.0)


In [ ]:
model.label_model.score

TypeError: score() missing 2 required positional arguments: 'L' and 'Y'

In [ ]:
model_lf_weights.values()

dict_values([0.9619468950526788, 0.9482166937299685, 0.9529086735518274, 0.9455221558078032, 0.9366248312982465, 0.9310878349063659, 0.9071245058144753, 0.9038895113266396, 0.8853151508392841, 0.840304193368001, 0.7668042545626503, 0.8577059641719714, 0.9044466808328667, 0.9332492060958133, 0.9849847811267727, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3603624881943688, 1.0, 1.0, 1.0, 1.0, 0.9917048030300066, 1.0, 0.5908339153858833, 1.0, 1.0, 1.0, 0.6248972205284535, 1.0, 1.0, 1.0, 1.0, 0.9539656884766494, 0.9170133598745451, 1.0, 1.0, 1.0, 0.9345297764626925, 0.9536078108552223, 0.9466952408230017, 0.9461321834950586, 0.782179660134716, 0.7408020378139173, 0.45792042749235407, 0.4861153755148394, 0.5126350680609485, 0.6515763225336797, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9929606605292064, 1.0, 0.9452091460577018, 0.7185830846683112, 0.46821456037979886, 0.557501698415917, 0.4430950541398488, 0.3709285137893156, 0.5336140704884504, 0.6

In [ ]:
cluster_manual = pickle.load(open("/workspace/datasets/clustering_manual_LFs_with_CLUSTERS.pkl", "rb"))

cluster_weights = {}
for v in set(cluster_manual.values()):
    cluster_weights[v] = []

for k in cluster_manual.keys():
    cluster_weights[cluster_manual[k]].append(model_lf_weights[k])

In [ ]:
for k in cluster_weights.keys():
    print(k, np.sum(cluster_weights[k]))

Pragmatic 2.8890252453093925
Structural 122.37321378481032
Syntactic 37.22355829608548
Lexical 108.92948228127669


In [ ]:
len(model_lf_weights.values())

328